# Tape Analysis

In [54]:
import pandas as pd
import datetime


In [55]:
df1 = pd.read_excel('input/cvx.xlsx') #import data from excel file
df1.sort_index(ascending=False, inplace = True) # rotate dataFrame
df1.drop(columns=['Bid','Ask','Sizes'],inplace=True) # delete columns
df1.dropna(how='any',inplace=True) # delete Nan Values

In [56]:
df1.index=df1['Date'] # set index as Date column

In [57]:
vol_filter =1000

In [58]:
df1

,Date,Time,Price,Size
Date,,,,
2014-03-11,2014-03-11,15:58:55,114.5900,77.0
2014-03-11,2014-03-11,15:58:55,114.5900,23.0
2014-03-11,2014-03-11,15:58:55,114.5900,1.0
2014-03-11,2014-03-11,15:58:55,114.6000,100.0
2014-03-11,2014-03-11,15:58:55,114.6000,100.0
...,...,...,...,...
2014-03-12,2014-03-12,13:17:22,115.5300,100.0
2014-03-12,2014-03-12,13:17:22,115.5300,200.0
2014-03-12,2014-03-12,13:17:22,115.5300,100.0


In [59]:
res_list =[]
pr = 0
dt = 0
tm = 0
sz = 0

ls1=[]

for index, row in df1.iterrows():  # itter every row in order to agregate size
    if (row['Price']== pr) & (row['Time']==tm) :
        sz += row['Size']
        
    if (row['Price']!= pr):
        
        ls1=(dt,tm,pr,sz)
        res_list.append(ls1)
        
        dt = row['Date']
        tm = row['Time']
        pr = row['Price']
        sz = row['Size']


In [93]:
res_list[1:]

[(Timestamp('2014-03-11 00:00:00'), datetime.time(15, 58, 55), 114.59, 101.0),
 (Timestamp('2014-03-11 00:00:00'), datetime.time(15, 58, 55), 114.6, 715.0),
 (Timestamp('2014-03-11 00:00:00'), datetime.time(15, 58, 55), 114.59, 52.0),
 (Timestamp('2014-03-11 00:00:00'), datetime.time(15, 58, 55), 114.6, 658.0),
 (Timestamp('2014-03-11 00:00:00'), datetime.time(15, 58, 55), 114.59, 200.0),
 (Timestamp('2014-03-11 00:00:00'), datetime.time(15, 58, 55), 114.6, 1000.0),
 (Timestamp('2014-03-11 00:00:00'), datetime.time(15, 58, 55), 114.59, 400.0),
 (Timestamp('2014-03-11 00:00:00'), datetime.time(15, 58, 55), 114.6, 100.0),
 (Timestamp('2014-03-11 00:00:00'), datetime.time(15, 58, 56), 114.61, 40.0),
 (Timestamp('2014-03-11 00:00:00'), datetime.time(15, 58, 56), 114.6, 180.0),
 (Timestamp('2014-03-11 00:00:00'), datetime.time(15, 58, 56), 114.61, 671.0),
 (Timestamp('2014-03-11 00:00:00'), datetime.time(15, 58, 56), 114.605, 18.0),
 (Timestamp('2014-03-11 00:00:00'), datetime.time(15, 58, 

In [94]:
# forming result dataSet
df_output = pd.DataFrame(res_list[1:])
df_output.columns= ['Date','Time','Price','Volume']
df_output['Price_1']=0
df_output['Volume_1']=0
df_output['Vol_filter']=0


In [95]:
pr = 0

for index, row in df_output.iterrows():
    if row['Price']> pr:
        
        df_output.at[index,'Price_1']=row['Price']
        df_output.at[index,'Volume_1']=row['Volume']
      
        
    if row['Price']< pr:
        df_output.at[index,'Price_1']=row['Price']*-1
        df_output.at[index,'Volume_1']=row['Volume']*-1
    pr = row['Price']   
    
    if row['Volume_1']>=vol_filter:
        df_output.at[index,'Vol_filter']= row['Volume_1']


In [63]:
df_output.loc['Volume_1'] = df_output.sum(numeric_only=True)

In [97]:
# using itertuples
for index, row in df_output.iterrows():
    if row['Volume_1']>=vol_filter:
        
        df_output.at[index,'Vol_filter']=row['Volume_1']
   
    

In [98]:
df_output

,Date,Time,Price,Volume,Price_1,Volume_1,Vol_filter
0,2014-03-11,15:58:55,114.590,101.0,114.590,101,0
1,2014-03-11,15:58:55,114.600,715.0,114.600,715,0
2,2014-03-11,15:58:55,114.590,52.0,-114.590,-52,0
3,2014-03-11,15:58:55,114.600,658.0,114.600,658,0
4,2014-03-11,15:58:55,114.590,200.0,-114.590,-200,0
...,...,...,...,...,...,...,...
11634,2014-03-12,13:17:12,115.518,100.0,-115.518,-100,0
11635,2014-03-12,13:17:15,115.520,400.0,115.520,400,0
11636,2014-03-12,13:17:15,115.515,100.0,-115.515,-100,0
11637,2014-03-12,13:17:15,115.520,300.0,115.520,300,0


In [65]:
#df['Time']=pd.to_datetime(df["Time"],format='%H:%M:%S')

In [100]:
with pd.ExcelWriter("output/output.xlsx") as writer:
   
    df_output.to_excel(writer, sheet_name="output")
    